In [3]:
import os
from gymhelpers import ExperimentsManager
import scipy.io as scipyio
import numpy as np
from collections import defaultdict

In [4]:
strategies = ["sparsemax","sparsemax","sparsemax"]
backuprules = ["sparsebellman","softbellman","bellman"]
temperatures = [1, 0.1, 0.01]
                #, 0.01, 0.01,0.1, 0.1,1,0.1]
temperatures_name = ["high","mid","low"]

action_res_list = [2001, 1001, 101, 3]
action_res_name = ["large","midlarge","midsmall","small"]

env_name = "InvertedPendulum-v1"
min_avg_rwd = 930
stop_training_min_avg_rwd = 950
layers_size = [512, 512]
n_ep = 1500
n_exps = 5

figures_dir = 'Figures'
api_key = '###'
alg_id = '###'

data = defaultdict(lambda : defaultdict(lambda : defaultdict(lambda : defaultdict(lambda : None))))
for action_res, action_name in zip(action_res_list,action_res_name):
    for temperature, temperature_name, strategy, backuprule in zip(temperatures, temperatures_name, strategies, backuprules):
        gym_stats_dir_prefix = os.path.join('Gym_stats', 
                                            env_name+'/'+str(strategy)+'/'+str(backuprule)+'/'+str(temperature_name)+'/'+str(action_name))
        print("Problem: {}, Actions: {}, Temp: {}, Strategy: {}, Backup: {}".format(env_name,np.prod(action_res),temperature,strategy,backuprule))
        expsman = ExperimentsManager(env_name=env_name, agent_value_function_hidden_layers_size=layers_size,
                             figures_dir=figures_dir, discount=0.99, decay_eps=0.995, eps_min=1E-4, learning_rate=3E-4,
                             decay_lr=True, max_step=2000, replay_memory_max_size=100000, ep_verbose=False,
                             exp_verbose=False, learning_rate_end=3E-5, batch_size=128, upload_last_exp=False, double_dqn=True, dueling=False,
                             target_params_update_period_steps=75, replay_period_steps=4, min_avg_rwd=min_avg_rwd,
                             per_proportional_prioritization=True, per_apply_importance_sampling=True, per_alpha=0.2,
                             per_beta0=0.4,
                             results_dir_prefix=gym_stats_dir_prefix, gym_api_key=api_key, gym_algorithm_id=alg_id,
                             strategy=strategy,backuprule=backuprule,
                             temperature_max=temperature, decay_temperature=1, temperature_min=temperature,
                             action_res=action_res,
                             video_recording=False)
        _, _, Rwd_per_ep_v, Loss_per_ep_v, Eval_ret = expsman.run_experiments(n_exps=n_exps, n_ep=n_ep, stop_training_min_avg_rwd=stop_training_min_avg_rwd, plot_results=False)
        data[action_name][temperature_name][strategy][backuprule]  = {"reward_list":Rwd_per_ep_v,"loss_list":Loss_per_ep_v,"Evaluation":Eval_ret}

scipyio.savemat(env_name+"_sparsemax_backup_with_best_temp.mat", data)
print("{} is finished and is saved".format(env_name))

[2017-11-20 23:54:09,266] Making new env: InvertedPendulum-v1
[2017-11-20 23:54:09,270] Making new env: InvertedPendulum-v1


Problem: InvertedPendulum-v1, Actions: 2001, Temp: 1, Strategy: sparsemax, Backup: sparsebellman

EXECUTING EXPERIMENT 0 OF 5 IN ENVIRONMENT InvertedPendulum-v1.


KeyboardInterrupt: 